## Импорты и проч

In [1]:
# Импортируем необходимые библиотеки и функции
import os
import json
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf

from baskerville import seqnn, gene as bgene
from borzoi_helpers import process_sequence, predict_tracks  # предполагается, что эти функции доступны

# Отключаем лишние предупреждения TensorFlow
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2025-02-14 16:24:25.925089: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 16:24:26.000849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-14 16:24:26.000915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-14 16:24:26.001026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 16:24:26.028413: I tensorflow/core/platform/cpu_feature_g

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
!nvcc -V

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.14.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## Препроцессинг

Код для понимания какой индекс отвечает за какую клеточную линию

In [3]:
import pandas as pd

targets_file = 'targets_gtex.txt'
targets_df = pd.read_csv(targets_file, sep='\t', index_col=0)

# Часто удобно иметь явный столбец индексов внутри датафрейма
targets_df['local_index'] = range(len(targets_df))

# Ткани, которые нас интересуют
tissue_list = [
    "kidney", 
    "liver",
    "adrenal",
    "pancreas",
    "lung"
]

print("Список каналов для нужных тканей:\n")

for tissue in tissue_list:
    # Поищем все строки, где в описании есть это слово/фраза (без учёта регистра)
    matches = targets_df[ targets_df['description'].str.lower().str.contains(tissue.lower()) ]
    
    if len(matches) == 0:
        print(f"Ткань '{tissue}' не найдена в targets_df['description'].")
    else:
        print(f"Ткань '{tissue}':")
        display(matches[['description','local_index']])
        print("-"*50)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Список каналов для нужных тканей:

Ткань 'kidney':


,description,local_index
7560,RNA:kidney,38
7561,RNA:kidney,39
7562,RNA:kidney,40


--------------------------------------------------
Ткань 'liver':


,description,local_index
7563,RNA:liver,41
7564,RNA:liver,42
7565,RNA:liver,43


--------------------------------------------------
Ткань 'adrenal':


...

--------------------------------------------------
Ткань 'pancreas':


,description,local_index
7577,RNA:pancreas,55
7578,RNA:pancreas,56
7579,RNA:pancreas,57


--------------------------------------------------
Ткань 'lung':


,description,local_index
7566,RNA:lung,44
7567,RNA:lung,45
7568,RNA:lung,46


--------------------------------------------------


In [64]:
targets_df

,identifier,file,clip,clip_soft,scale,sum_stat,strand_pair,description
7522,GTEX-132QS-2526-SM-62LFJ.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7522,RNA:adipose_tissue
7523,GTEX-1GMR3-0826-SM-9WYT4.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7523,RNA:adipose_tissue
7524,GTEX-1HSEH-0226-SM-ACKVV.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7524,RNA:adipose_tissue
7525,GTEX-11GSP-0326-SM-5A5KW.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7525,RNA:adrenal_gland
7526,GTEX-13PVR-0226-SM-5RQJI.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7526,RNA:adrenal_gland
...,...,...,...,...,...,...,...,...
7606,GTEX-13FTX-1026-SM-5J2O5.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7606,RNA:uterus
7607,GTEX-1MA7W-1526-SM-DHXKS.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7607,RNA:uterus
7608,GTEX-11EMC-1926-SM-5A5JU.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7608,RNA:vagina
7609,GTEX-12WSB-2426-SM-5EGJC.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.01,sum_sqrt,7609,RNA:vagina


In [3]:
kidney_ix =  39  
liver_ix =   42  # "left lobe of liver tissue"
adrenal_ix = 4
pancreas_ix =56
lung_ix =    45

# Словарь: название ткани -> индекс канала модели
channels_dict = {
    "ENCFF123KIW_kidney"                 : kidney_ix,
    "ENCFF784MDF_left_lobe_of_liver"     : liver_ix,
    "ENCFF236XOK_adrenal_gland"          : adrenal_ix,
    "ENCFF781TTC_pancreas"               : pancreas_ix,
    "ENCFF242BWW_lung"                   : lung_ix
}

In [17]:
import pandas as pd
import gzip

# Загрузим файл .bed.gz с последовательностями
bed_file_path = '../data/sequences_human.bed.gz'

# Открываем и загружаем данные из .bed файла
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, names=["chrom", "start", "end", "fold"])

# Фильтрация данных по фолду
folds_to_process = ['fold3']  # Можно изменить на нужные фолды
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Сгруппируем по хромосомам
grouped = filtered_bed_data.groupby("chrom")

print(len(filtered_bed_data))
# Посмотрим первые несколько строк
filtered_bed_data.head()

6888


,chrom,start,end,fold
20329,chr5,41626145,41822753,fold3
20330,chr11,40389266,40585874,fold3
20331,chr11,32570759,32767367,fold3
20332,chr6,164265822,164462430,fold3
20333,chr5,13736747,13933355,fold3


In [58]:
from tqdm import tqdm

def merge_intervals(intervals):
    """
    Принимает список интервалов вида [(start, end), ...] и возвращает список объединённых интервалов.
    """
    # Сортируем интервалы по start
    intervals.sort(key=lambda x: x[0])
    merged = []
    for start, end in intervals:
        if not merged or start > merged[-1][1]:
            merged.append([start, end])
        else:
            merged[-1][1] = max(merged[-1][1], end)
    return merged

print("Количество интервалов до мерджа:", len(filtered_bed_data))

# Объединяем интервалы по каждой хромосоме с прогресс-баром.
merged_intervals_list = []
# Количество групп = число уникальных хромосом в filtered_bed_data
unique_chroms = filtered_bed_data['chrom'].nunique()
for chrom, group in tqdm(filtered_bed_data.groupby("chrom"), desc="Мержим интервалы", total=unique_chroms):
    # Получаем список интервалов для данной хромосомы
    intervals = group[['start', 'end']].values.tolist()
    merged = merge_intervals(intervals)
    # Если все интервалы имеют одинаковый fold, то просто берем первый
    fold_val = group['fold'].iloc[0]
    for start, end in merged:
        merged_intervals_list.append({'chrom': chrom, 'start': start, 'end': end, 'fold': fold_val})

# Перезаписываем filtered_bed_data объединёнными интервалами
filtered_bed_data = pd.DataFrame(merged_intervals_list)

# Выведем результат для проверки
print("Количество интервалов после мерджа:", len(filtered_bed_data))
filtered_bed_data.head()


Количество интервалов до мерджа: 6888


Мержим интервалы: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 918.55it/s]

Количество интервалов после мерджа: 46


,chrom,start,end,fold
0,chr1,143479625,143823752,fold3
1,chr1,227321006,228550247,fold3
2,chr11,25440674,48748592,fold3
3,chr11,48896195,49191149,fold3
4,chr11,54525074,54820028,fold3


## Проверка данных

Убедимся что все записи влезут во входящее окно модели

In [59]:
# Находим максимальную длину последовательности
max_sequence_length = 0

for index, row in filtered_bed_data.iterrows():
    start = row['start']
    end = row['end']
    
    # Вычисляем длину текущей последовательности
    sequence_length = end - start
    
    # Обновляем максимальную длину, если текущая больше
    if sequence_length > max_sequence_length:
        max_sequence_length = sequence_length

print(f"Максимальная длина последовательности: {max_sequence_length} нуклеотидов")


Максимальная длина последовательности: 67957002 нуклеотидов


Проверка на пересечения

In [60]:
import pandas as pd
from tqdm import tqdm

overlap_found = False

# Получаем список уникальных хромосом
chromosomes = filtered_bed_data['chrom'].unique()

# Используем tqdm для отображения прогресса
for chrom in tqdm(chromosomes, desc="Checking for overlaps"):
    # Фильтруем по текущей хромосоме
    group = filtered_bed_data[filtered_bed_data['chrom'] == chrom]
    
    # Сортируем записи по старту
    sorted_group = group.sort_values('start')
    
    # Инициализируем предыдущую запись
    prev_row = None
    
    # Проходим по отсортированным записям
    for idx, row in sorted_group.iterrows():
        if prev_row is not None:
            # Проверяем пересечение
            if row['start'] < prev_row['end']:
                print(f"Пересечение на {chrom}: {prev_row[['start', 'end']].to_dict()} и {row[['start', 'end']].to_dict()}")
                overlap_found = True
        prev_row = row

if not overlap_found:
    print("Пересечений не обнаружено в filtered_bed_data.")


Checking for overlaps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1091.60it/s]

Пересечений не обнаружено в filtered_bed_data.


In [61]:
import pandas as pd
import gzip

# Загрузим данные из bed-файла
bed_file_path = '../data/sequences_human.bed.gz'
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, 
                           names=["chrom", "start", "end", "fold"])

# Фильтруем данные по fold3 и fold4
#folds_to_process = ['fold3', 'fold4']
folds_to_process = ['fold3']
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Выведем список уникальных хромосом
unique_chroms = filtered_bed_data['chrom'].unique()
print("Уникальные хромосомы в выборке:")
print(unique_chroms)
print(f"Количество уникальных хромосом: {len(unique_chroms)}")

# Для каждой хромосомы находим минимальный start и максимальный end
chrom_stats = filtered_bed_data.groupby("chrom").agg(min_start=('start', 'min'),
                                                      max_end=('end', 'max'))
print("\nМинимальный start и максимальный end по хромосомам:")
print(chrom_stats)

if len(unique_chroms) > 1:
    print("\nВыборка содержит более одной хромосомы.")
else:
    print("\nВыборка содержит только одну хромосому.")


Уникальные хромосомы в выборке:
['chr5' 'chr11' 'chr6' 'chr9' 'chr13' 'chr2' 'chr8' 'chr7' 'chr17' 'chrX'
 'chr18' 'chr12' 'chr15' 'chr19' 'chr1' 'chr16' 'chr20']
Количество уникальных хромосом: 17

Минимальный start и максимальный end по хромосомам:
       min_start    max_end
chrom                      
chr1   143479625  228550247
chr11   25440674   56541083
chr12   32992234   34368994
chr13   40653298  102135774
chr15   20168638   34675550
chr16   33491584   33884884
chr17   81799133   83225066
chr18   13485786   15206757
chr19    3108622    9156817
chr2    68747521  129505661
chr20   30186694   30383302
chr5     8770274  103148800
chr6    99603327  170690035
chr7       10000   74778699
chr8     8158857   55824055
chr9    80588528  138217638
chrX     3230043   58109050

Выборка содержит более одной хромосомы.


# Инференс

In [12]:
import os

# Предполагается, что переменные output_dir, folds_to_process и channels_dict уже определены
output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

# Собираем строку из folds_to_process, например: "fold3" или "fold3_fold4"
folds_str = "_".join(folds_to_process)

# Для каждой ткани создаём по два файла: для прямой (st+) и для обратной (st-) цепи.
chains = ["st+", "st-"]

for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        with open(file_path, "w") as f:
            # Опционально можно записать заголовок трека, если потребуется для визуализации
            f.write(f"track type=bedGraph name=\"{bed_name} {chain}\"\n")

print("Предсозданы файлы:")
for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        print(file_path)


Предсозданы файлы:
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF242BWW_lung.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF242BWW_lung.bedGraph


In [18]:
center_len = 196_000
context_len = 160_000

# Загрузим FASTA
import pysam
fasta_path = "hg38/assembly/ucsc/hg38.fa"
fasta_index = pysam.Fastafile(fasta_path)

# Получим размеры хромосом
chr_sizes = {chrom: fasta_index.get_reference_length(chrom) for chrom in fasta_index.references}

# Отсортируем хромосомы в "естественном" порядке
def chr_order_key(chrom_name):
    name = chrom_name.replace("chr", "")
    if name.isdigit():
        return (int(name), "")
    elif name == "X":
        return (9999998, "X")
    elif name == "Y":
        return (9999999, "Y")
    elif name in ["M","MT"]:
        return (10000000, "M")
    else:
        return (20000000, name)

chrom_list_sorted = sorted(chr_sizes.keys(), key=chr_order_key)
chrom_list_filtered = [c for c in chrom_list_sorted if c in grouped.groups.keys()]

# Собираем все окна в один список
all_windows = []
for chrom in chrom_list_filtered:
    group = grouped.get_group(chrom)
    region_start = group["start"].min()
    region_end   = group["end"].max()

    # Разбиваем [region_start..region_end] на окна
    curr_start = region_start
    while curr_start < region_end:
        curr_end = curr_start + center_len
        if curr_end > region_end:
            curr_end = region_end
        all_windows.append((chrom, curr_start, curr_end))
        curr_start = curr_end

# Опционально сортируем all_windows глобально (сначала по хромосоме, потом по start)
def window_sort_key(item):
    c, st, en = item
    return (chr_order_key(c), st)

all_windows.sort(key=window_sort_key)

print(f"Total windows to process: {len(all_windows)}")


Total windows to process: 3400


In [36]:
import numpy as np
import time
from tqdm import tqdm
import os

stride = models[0].model_strides[0]

def reverse_complement_onehot(seq_onehot):
    """
    Для массива seq_onehot формы (L, 4),
    где порядок A=0, C=1, G=2, T=3.
    Возвращаем реверс-комплементарную последовательность того же размера (L,4).
    """
    rev = np.flip(seq_onehot, axis=0)
    revcomp = rev[:, [3, 2, 1, 0]]
    return revcomp

def process_window(chrom, center_start, center_end, context_len, models):
    """
    Выполняем:
      1) Добавляем контекст к центральному окну.
      2) Получаем one-hot для + цепи и предсказываем y_pred_plus.
      3) Получаем реверс-комплемент one-hot для − цепи и предсказываем y_pred_minus.
      4) Переводим координаты центрального окна (в нуклеотидах) в индексы выхода (делим на stride).
      5) Возвращаем (center_pred_plus, center_pred_minus) формы (L_out, C).
    """
    # Определяем входной интервал с контекстом
    input_start = max(0, center_start - context_len)
    input_end   = min(chr_sizes[chrom], center_end + context_len)

    # Получаем one-hot для + цепи
    seq_one_hot_plus = process_sequence(fasta_index, chrom, input_start, input_end)
    
    # Предсказываем + цепь
    y_pred_plus = predict_tracks(models, seq_one_hot_plus)
    y_pred_plus = np.squeeze(y_pred_plus)  # ожидается (L_out, C)

    # Получаем one-hot для − цепи (реверс-комплемент)
    seq_one_hot_minus = reverse_complement_onehot(seq_one_hot_plus)
    y_pred_minus = predict_tracks(models, seq_one_hot_minus)
    y_pred_minus = np.squeeze(y_pred_minus)

    # Переводим координаты центрального окна в индексы выхода
    offset = center_start - input_start
    offset_out = offset // stride

    center_len_actual = center_end - center_start
    center_len_actual_out = center_len_actual // stride

    # Проверки на корректность
    if center_len_actual_out <= 0:
        return None, None

    L_plus = y_pred_plus.shape[0]
    if offset_out < 0:
        offset_out = 0
    if offset_out + center_len_actual_out > L_plus:
        center_len_actual_out = L_plus - offset_out
    if center_len_actual_out <= 0:
        return None, None

    center_pred_plus = y_pred_plus[offset_out : offset_out + center_len_actual_out, :]

    L_minus = y_pred_minus.shape[0]
    if offset_out + center_len_actual_out > L_minus:
        center_len_actual_out = L_minus - offset_out
    if center_len_actual_out <= 0:
        return None, None

    center_pred_minus = y_pred_minus[offset_out : offset_out + center_len_actual_out, :]

    return center_pred_plus, center_pred_minus

def run_inference_all_windows(all_windows, models):
    """
    Проходим по всем окнам (единый цикл).
    Для каждого окна делаем:
      - Получаем center_pred_plus и center_pred_minus.
      - Записываем результаты в соответствующие bedGraph-файлы.
    Файлы открываем в режиме 'a' (append), сразу пишем и закрываем, чтобы видеть рост файлов в реальном времени.
    """
    start_time = time.time()
    print(f"Running inference on {len(all_windows)} windows (plus & minus).")

    # Сформируем строку о фолдах
    folds_str = "_".join(folds_to_process)

    for (chrom, center_start, center_end) in tqdm(all_windows, desc="Global windows"):
        center_pred_plus, center_pred_minus = process_window(chrom, center_start, center_end, context_len, models)
        if center_pred_plus is None or center_pred_minus is None:
            # Пропускаем окно, если предсказания отсутствуют
            continue
        
        # Записываем для каждого канала
        for bed_name, channel_ix in channels_dict.items():
            # Если индекс канала выходит за пределы предсказаний, пропускаем
            if channel_ix >= center_pred_plus.shape[1]:
                continue

            # Для + цепи
            channel_values_plus = center_pred_plus[:, channel_ix]
            file_path_plus = os.path.join(
                output_dir,
                f"borzoi_rnaseq_{folds_str}_st+_{bed_name}.bedGraph"
            )
            with open(file_path_plus, "a") as f_plus:
                for i, val in enumerate(channel_values_plus):
                    pos_start = center_start + i  # координаты в нуклеотидах остаются без преобразования
                    pos_end = pos_start + 1
                    if pos_end <= pos_start:
                        continue
                    f_plus.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

            # Для − цепи
            if channel_ix >= center_pred_minus.shape[1]:
                continue
            channel_values_minus = center_pred_minus[:, channel_ix]
            file_path_minus = os.path.join(
                output_dir,
                f"borzoi_rnaseq_{folds_str}_st-_{bed_name}.bedGraph"
            )
            with open(file_path_minus, "a") as f_minus:
                for i, val in enumerate(channel_values_minus):
                    pos_start = center_start + i
                    pos_end = pos_start + 1
                    if pos_end <= pos_start:
                        continue
                    f_minus.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

    elapsed = time.time() - start_time
    print(f"Inference completed in {elapsed:.2f} seconds total.")


In [38]:
print("Start inference...")

# Запускаем
run_inference_all_windows(all_windows, models)

print("All done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.")


Start inference...
Running inference on 3400 windows (plus & minus).


Global windows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3400/3400 [1:55:55<00:00,  2.05s/it]

Inference completed in 6955.56 seconds total.
All done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.
